##### Začneme tím, že naimportujeme knihovny:

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

##### V dalším kroku budeme stahovat soubory od roku 2018 z webu SÚKL a postupně budeme přidávat starší soubory.

In [2]:
response = requests.get('http://www.sukl.cz/2018')

In [3]:
soup = BeautifulSoup(response.text, "lxml")

Odkazy v HTML začínají písmenkem "a" a mají atribut "href".  Zároveň na stránce http://www.sukl.cz/2018 vidíme, že názvy složek začínají slovem "Distribuce":

In [4]:
links = soup.find_all('a')

In [5]:
for slovo in links:
    
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href'] #Po několika pokusech jsme zjistily, že potřebujeme přidat 'http://www.sukl.cz'
        print(x) #Vytiskneme každý odkaz, abychom mohly prověřit, jestli opravdu fungují

http://www.sukl.cz/file/87404_1_1
http://www.sukl.cz/file/87628_1_1
http://www.sukl.cz/file/87831_1_1


##### Vytvoříme seznam "our_list" a nahrajeme do něj odpovídající odkazy:

In [6]:
our_list = ['']
i = 0
for slovo in links:
    if slovo.text.startswith('DistribuceLP_'):
        x ='http://www.sukl.cz' + slovo.attrs['href']
        if i == 0:
            our_list[i] = x
        else:
            our_list.append(x)
        i+=1
print(our_list)

['http://www.sukl.cz/file/87404_1_1', 'http://www.sukl.cz/file/87628_1_1', 'http://www.sukl.cz/file/87831_1_1']


##### Vytvoříme seznam "our_files", do kterého nahrajeme každý ze souborů s pomocí seznamu odkazů:

In [7]:
our_files = [ pd.read_excel(x, encoding='utf-8') for x in our_list ]


##### Abychom mohly prověřit, jestli je všechno jak má být, vypíšeme počet řádků v každém ze souborů a pak jejich součet:

In [8]:
number_of_rows =[0,0,0] #Vidíme, že v roce 2018 máme jen 3 soubory. Pro předchozí roky budeme používat seznam obsahující 12 čísel (protože 12 měsíců)

for i in range(0, len(our_files)):
    number_of_rows[i] = our_files[i].shape[0]
number_of_rows

[24090, 23931, 24133]

In [9]:
sum(number_of_rows) 

72154

##### Teď konečně spojíme složky. Víme, že mají stejnou strukturu (počet a názvy sloupců), proto nahrajeme soubory jeden za druhým. Budeme postupovat podle vzoru:

year_201x = pd.concat([our_files[0], our_files[1], "a tak dale" ], ignore_index = True) 

Samozřejmě pro rok 2018 není potřeba psát cyklus, ale bude se hodit pro ostatní roky, kde máme více složek:

In [10]:
for i in range(1, len(our_files)):
    our_files[i] = pd.concat([our_files[i-1], our_files[i]], ignore_index = True)
year_2018 = our_files[i]
year_2018.head()

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,2018.01,DISTRIBUTOR_CR,A01A,53907.0,SEPTOMIXINE,"286MG/G+190000IU/G STM PST 7,5G",SPH,F,D,15.0,NaN,NaN,NaN,NaN,R
1,2018.01,LEKARNA,A01A,53907.0,SEPTOMIXINE,"286MG/G+190000IU/G STM PST 7,5G",SPH,F,D,53.0,1157.554340,61350.38,NaN,NaN,R
2,2018.01,DISTRIBUTOR_CR,A01AA01,208774.0,ZYMAFLUOR,"0,25MG TBL NOB 200",MHS,CZ,D,2359.0,NaN,NaN,100.0,235900.0,F
3,2018.01,LEKARNA,A01AA01,208774.0,ZYMAFLUOR,"0,25MG TBL NOB 200",MHS,CZ,D,1715.0,94.806134,162592.52,100.0,171500.0,F
4,2018.01,LEKARNA,A01AA01,208774.0,ZYMAFLUOR,"0,25MG TBL NOB 200",MHS,CZ,V,-8.0,94.800000,-758.40,100.0,-800.0,F


Zkontrolujeme, jestli je počet řádků stejný, jako v "sum(number_of_rows)", což dělá 72154 řádků pro rok 2018

In [11]:
year_2018.shape

(72154, 15)

##### Hurá! Tuto část máme hotovou. Stejným způsobem tak můžeme vytvořit tabulky "year_2017" až po "year_2011":

In [12]:
year_2018.to_csv('years/year_2018.csv',  encoding='utf-8')

Až budeme mít hotovou tabulku za každý rok, spojíme je jedním příkazem dohromady.